In [ ]:
import os
import geopandas as gpd
import pickle
from datetime import date
from tqdm import tqdm

tqdm.pandas()

SHARED_PROJECT_PATH = '...'

In [ ]:
today = date.today().strftime('%d-%m-%Y')
out_path = os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today)

gdf_graph_edges_conn_undir = gpd.read_file(os.path.join(out_path, f'GDF_BP-network-conn-undir_{today}.json'))
graph_bp_edges_dir = pickle.load(open(os.path.join(out_path, f'GNX_BP-network-conn-dir_{today}.json'), 'rb'))

---

## All graph edges have all attributes - e.g. link attribute is there

In [ ]:
graph_bp_edges_dir.edges[(9515749, 502897904, 0)]

In [ ]:
gdf_graph_edges_conn_undir.head()

---

## All graph edges CAN BE and GET projected to gdf

### No duplicate edge ids in graph

In [ ]:
graph_edges = list(graph_bp_edges_dir.edges)

In [ ]:
graph_edges_set = set(graph_edges)

In [ ]:
assert len(graph_edges) == len(graph_edges_set)

### All dir graph ids have undir gdf id pairs

In [ ]:
def gnx_edge_id_str_to_tuple(s):
    l = s[1:-1].split(', ')
    for i in range(len(l)):
        try:
            l[i] = int(l[i])
        except:
            l[i] = l[i][1:-1]
    return tuple(l)

In [ ]:
gdf_edges = gdf_graph_edges_conn_undir['gnx_edge_id'].apply(gnx_edge_id_str_to_tuple).to_list() +\
            gdf_graph_edges_conn_undir[~gdf_graph_edges_conn_undir['oneway']]['gnx_edge_id'].apply(gnx_edge_id_str_to_tuple).apply(lambda t: (t[1], t[0], t[2])).to_list()
gdf_edges_set = set(gdf_edges)

In [ ]:
assert len(gdf_edges_set.intersection(graph_edges_set)) == len(gdf_edges_set)

### Duplicates - circular road segments (origin = destination)

In [ ]:
gdf_all_set = set(gdf_graph_edges_conn_undir['gnx_edge_id'].apply(gnx_edge_id_str_to_tuple).to_list())
gdf_undir_rev_set = set(gdf_graph_edges_conn_undir[~gdf_graph_edges_conn_undir['oneway']]['gnx_edge_id'].apply(gnx_edge_id_str_to_tuple).apply(lambda t: (t[1], t[0], t[2])).to_list())
len(gdf_all_set.intersection(gdf_undir_rev_set))

In [ ]:
gdf_graph_edges_conn_undir[gdf_graph_edges_conn_undir['gnx_edge_id'].isin(list(map(str, gdf_all_set.intersection(gdf_undir_rev_set))))]

### All dir graph id - undir gdf id pairs match (geometries)

- oneway: same id both in gdf and graph
- twoway: gdf id and inverse [(to, from, key)] in graph

In [ ]:
def verify_matching_geometries(edge_id, oneway):
    t = gnx_edge_id_str_to_tuple(edge_id)
    graph_geom = graph_bp_edges_dir.edges[t]['geometry']
    gdf_geom = gdf_graph_edges_conn_undir[gdf_graph_edges_conn_undir['gnx_edge_id'] == edge_id].iloc[0].geometry
    if oneway:
        return (gdf_geom == graph_geom)  # geometry match
    else:
        t_rev = (t[1], t[0], t[2])
        graph_geom_rev = graph_bp_edges_dir.edges[t_rev]['geometry']
        return ((gdf_geom == graph_geom) & (gdf_geom.reverse() == graph_geom_rev)) | (t == t_rev)   # geometry + reverse geometry match or circular segment

In [ ]:
filt = gdf_graph_edges_conn_undir.progress_apply(lambda row: verify_matching_geometries(row['gnx_edge_id'], row['oneway']), axis=1)

In [ ]:
assert gdf_graph_edges_conn_undir[~filt].shape[0] == 0